## Load the required packages

In [ ]:
%pip install --upgrade numexpr --quiet

%pip install pandas --quiet
%pip install datetime --quiet
%pip install feedparser --quiet
%pip install textblob --quiet
%pip install yfinance --quiet
%pip install requests --quiet

## Define a function to get the pricing data

In [ ]:
import pandas as pd
import yfinance as yf
from typing import List, Tuple

def fetch_price(ticker: str,
               period: str, 
               interval: str) -> pd.DataFrame:
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=True, progress=False)
    df.columns = df.columns.get_level_values(0)
    df.reset_index(inplace=True) 
    df.columns.name = None
    df['ticker'] = ticker
    return df

## Load configuration

In [ ]:
import sys

# Standard imports
import yaml
from pathlib import Path

# Import from nadex_common package
from nadex_common.strategy_rsi import (
    generate_rsi_signals, 
    rsi_wilder, 
    macd, 
    apply_guardrails,
    calculate_signal_confidence
)
from nadex_common.utils_s3 import (
    create_s3_clients,
    get_bucket,
    upload_df_to_s3_with_validation,
    append_runlog_s3,
    save_dataframe_to_s3,
    assert_allowed_bucket
)

# Load configs
with open('../configs/s3.yaml', 'r') as f:
    s3_cfg = yaml.safe_load(f)

with open('../configs/strategy.yaml', 'r') as f:
    strategy_cfg = yaml.safe_load(f)

with open('../configs/ticker_mappings.yaml', 'r') as f:
    tickers_cfg = yaml.safe_load(f)

# Set variables
BUCKET = s3_cfg['bucket']
REGION = s3_cfg.get('region')
RECS_PREFIX = s3_cfg['prefixes'].get('recommendations', 'recommendations')
REPORTS_PREFIX = s3_cfg['prefixes'].get('reports', 'reports')
RUNLOG_KEY = f"{s3_cfg['prefixes'].get('logs','logs')}/run_log.csv"

print(f"✅ Loaded S3 config:")
print(f"   Bucket: {BUCKET}")
print(f"   Region: {REGION}")

print(f"\n✅ Loaded Strategy config:")
print(f"   RSI Period: {strategy_cfg['rsi']['period']}")
print(f"   RSI Mode: {strategy_cfg['rsi']['mode']}")
print(f"   RSI Centerline: {strategy_cfg['rsi']['centerline']}")
print(f"   RSI Oversold: {strategy_cfg['rsi']['oversold']}")
print(f"   RSI Overbought: {strategy_cfg['rsi']['overbought']}")
print(f"   Trend Type: {strategy_cfg['trend']['type']}")
print(f"   MACD: {strategy_cfg['trend']['macd_fast']}/{strategy_cfg['trend']['macd_slow']}/{strategy_cfg['trend']['macd_signal']}")
print(f"\n✅ Loaded Guardrails:")
print(f"   Confidence Threshold: {strategy_cfg['guardrails']['confidence_threshold']}")
print(f"   Max Positions/Day: {strategy_cfg['guardrails']['max_positions_per_day']}")

# Calculate ticker counts from config
all_tickers = tickers_cfg.get('tickers', {})
active_tickers = [t for t, info in all_tickers.items() if info.get('active', False)]
test_tickers = tickers_cfg.get('test_tickers', [])

print(f"\n✅ Loaded Tickers config:")
print(f"   Total tickers in system: {len(all_tickers)}")
print(f"   Active trading tickers: {len(active_tickers)}")
print(f"   Test tickers: {len(test_tickers)}")

MAPPING_FILE = Path(s3_cfg.get('mapping_file')).resolve() if s3_cfg.get('mapping_file') else None
ALLOWED_BUCKETS = {BUCKET}

import sys
print("\n✅ Successfully imported nadex_common modules")


## Define a test strategy

In [ ]:
import copy

def test_strategy_modes():
    """Test both centerline and reversal RSI modes."""
    import copy
    
    # Create test data
    test_prices = pd.Series([100, 102, 104, 103, 101, 99, 98, 97, 96, 95,
                             94, 95, 96, 98, 100, 102, 104, 106, 108, 110])
    
    # Test 1: Centerline mode
    cfg_centerline = copy.deepcopy(strategy_cfg)
    cfg_centerline['rsi']['mode'] = 'centerline'
    cfg_centerline['rsi']['centerline'] = 50
    
    signals_cl = generate_rsi_signals(test_prices, cfg_centerline)
    print("✅ Centerline mode test:")
    print(f"   Signals generated: {len(signals_cl)}")
    print(f"   Buy signals: {(signals_cl['signal'] == 1).sum()}")
    print(f"   Sell signals: {(signals_cl['signal'] == -1).sum()}")
    print(f"   No trade: {(signals_cl['signal'] == 0).sum()}")
    
    # Test 2: Reversal mode
    cfg_reversal = copy.deepcopy(strategy_cfg)
    cfg_reversal['rsi']['mode'] = 'reversal'
    cfg_reversal['rsi']['oversold'] = 30
    cfg_reversal['rsi']['overbought'] = 70
    cfg_reversal['rsi']['require_cross'] = True
    
    signals_rv = generate_rsi_signals(test_prices, cfg_reversal)
    print("\n✅ Reversal mode test:")
    print(f"   Signals generated: {len(signals_rv)}")
    print(f"   Buy signals: {(signals_rv['signal'] == 1).sum()}")
    print(f"   Sell signals: {(signals_rv['signal'] == -1).sum()}")
    print(f"   No trade: {(signals_rv['signal'] == 0).sum()}")
    
    # Test 3: Guardrails
    test_signals_df = pd.DataFrame({
        'Recommendation': ['Buy', 'Sell', 'Buy', 'Buy', 'Sell'],
        'Confidence': [0.8, 0.7, 0.65, 0.55, 0.9],
        'Ticker': ['ES=F', 'NQ=F', 'GC=F', 'CL=F', 'YM=F']
    })
    
    filtered = apply_guardrails(test_signals_df, strategy_cfg)
    trade_count = len(filtered[filtered['Recommendation'] != 'No trade'])
    
    print(f"\n✅ Guardrails test:")
    print(f"   Original signals: {len(test_signals_df)}")
    print(f"   After guardrails: {trade_count}")
    print(f"   Max positions: {strategy_cfg['guardrails']['max_positions_per_day']}")
    print(f"   Confidence threshold: {strategy_cfg['guardrails']['confidence_threshold']}")
    
    return True

# Run tests
test_strategy_modes()

## Compute the technical indicators
1. Compute the MACD
2. Compute the RSI
3. Compute the ATR

In [ ]:
import pandas as pd
from nadex_common.strategy_rsi import (
    generate_rsi_signals, 
    rsi_wilder, 
    macd, 
    apply_guardrails,
    calculate_signal_confidence
)

def compute_indicators(df: pd.DataFrame, strategy_cfg: dict) -> pd.DataFrame:
    """
    Compute technical indicators using strategy.yaml configuration.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with OHLC data
    strategy_cfg : dict
        Strategy configuration from strategy.yaml
        
    Returns
    -------
    pd.DataFrame with indicators added
    """
    df = df.copy()
    
    # Get config values
    rsi_period = strategy_cfg['rsi']['period']
    macd_fast = strategy_cfg['trend']['macd_fast']
    macd_slow = strategy_cfg['trend']['macd_slow']
    macd_signal = strategy_cfg['trend']['macd_signal']
    
    # Compute RSI using strategy_rsi.py function
    df['RSI'] = rsi_wilder(df['Close'], period=rsi_period)
    
    # Compute MACD using strategy_rsi.py function
    df['MACD'], df['Signal'], df['MACD_hist'] = macd(
        df['Close'], 
        fast=macd_fast, 
        slow=macd_slow, 
        signal=macd_signal
    )
    
    # Keep EMA calculations for compatibility
    df['EMA12'] = df['Close'].ewm(span=macd_fast, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=macd_slow, adjust=False).mean()
    
    # Compute ATR (not in strategy_rsi.py yet, keep custom)
    prev_close = df['Close'].shift(1)
    tr1 = df['High'] - df['Low']
    tr2 = (df['High'] - prev_close).abs()
    tr3 = (df['Low'] - prev_close).abs()
    df['ATR'] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1).rolling(window=14).mean()
    
    return df

## Define a function to load the strike prices

In [ ]:
import pandas as pd

def load_strikes(
    path: str,
    tickers_cfg: dict,
    strike_col: str = "strike",
    decimals: int = 4
) -> pd.DataFrame:
    """
    Load strike prices from nadex_contracts.csv and map instrument names to tickers.
    
    Parameters
    ----------
    path : str
        Path to the CSV file (nadex_contracts.csv format)
    tickers_cfg : dict
        Ticker configuration with mappings from ticker_mappings.yaml
    strike_col : str
        Name of the strike column in output DataFrame
    decimals : int
        Number of decimal places for strike prices
        
    Returns
    -------
    pd.DataFrame with columns: ticker, strike, expire
    """
    # Read the nadex_contracts CSV
    df = pd.read_csv(path)
    
    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()
    
    # Build mapping from Nadex instrument names to tickers using config
    # Map from description field in tickers_cfg
    nadex_name_map = {}
    
    # Common Nadex name variations mapped to their config descriptions
    nadex_to_description = {
        'US 500': 'E-mini S&P 500',
        'US Tech 100': 'E-mini NASDAQ 100',
        'Wall St 30': 'E-mini Dow Jones',
        'US SmallCap 2000': 'E-mini Russell 2000',
        'Crude Oil': 'Crude Oil',
        'Gold': 'Gold',
        'Natural Gas': 'Natural Gas',
        'EUR/USD': 'Euro / US Dollar',
        'USD/JPY': 'US Dollar / Japanese Yen',
        'AUD/USD': 'Australian Dollar / US Dollar',
        'GBP/USD': 'British Pound / US Dollar',
        'GBP/JPY': 'British Pound / Japanese Yen',
        'EUR/JPY': 'Euro / Japanese Yen'
    }
    
    # Build reverse mapping: description -> ticker from config
    description_to_ticker = {
        info.get('description'): ticker 
        for ticker, info in tickers_cfg.get('tickers', {}).items()
        if 'description' in info
    }
    
    # Create final mapping: Nadex name -> ticker
    for nadex_name, description in nadex_to_description.items():
        if description in description_to_ticker:
            nadex_name_map[nadex_name] = description_to_ticker[description]
    
    # Map instrument names to tickers
    df['ticker'] = df['instrument'].map(nadex_name_map)
    
    # Filter out any unmapped instruments
    unmapped = df[df['ticker'].isna()]['instrument'].unique()
    if len(unmapped) > 0:
        print(f"⚠️  Warning: {len(unmapped)} unmapped instruments found: {list(unmapped)}")
        df = df[df['ticker'].notna()]
    
    # Rename strike_price to strike
    df = df.rename(columns={'strike_price': strike_col})
    
    # Ensure strike column is numeric and rounded
    df[strike_col] = pd.to_numeric(
        df[strike_col].astype(str).str.strip(),
        errors='coerce'
    ).round(decimals)
    
    # Create expire column from date and expiration_time
    df['expire'] = df['date']
    
    # Select only the columns needed by the rest of the code
    result = df[['ticker', strike_col, 'expire']].copy()
    
    # Drop any rows with missing values
    result = result.dropna()
    
    return result


## Define a function to determine the trading signal -- RSI Mode Guide

### Centerline Mode (mode: centerline)
- **Use when:** Trading with the trend
- **Logic:** Buy when RSI > 50 and trend is up; Sell when RSI < 50 and trend is down
- **Best for:** Trending markets, momentum following
- **Parameters:** centerline (default: 50)

### Reversal Mode (mode: reversal)
- **Use when:** Trading reversals at extremes
- **Logic:** Buy when RSI crosses above oversold level; Sell when RSI crosses below overbought level
- **Best for:** Range-bound markets, mean reversion strategies
- **Parameters:** 
  - oversold (default: 30)
  - overbought (default: 70)
  - require_cross (default: true) - wait for RSI to cross back through threshold

### Trend Filter
- Applies to both modes
- Options: none, macd, or sma
- When type: macd, only takes buy signals when MACD > Signal (uptrend)
- When type: sma, only takes buy signals when price > SMA (uptrend)

### Guardrails
- confidence_threshold: Minimum confidence score (0.0-1.0) to generate recommendation
- max_positions_per_day: Maximum number of recommendations to output per day

In [ ]:
import pandas as pd

def signal_detail_for_row(row, per_ticker, strategy_cfg, expiry="EOD"):
    """
    Generate signal details for a single row, using strategy_rsi.generate_rsi_signals().
    """
    df = per_ticker[row.ticker]
    last = df.iloc[-1]
    
    # Use generate_rsi_signals from strategy_rsi.py
    signals = generate_rsi_signals(df['Close'], strategy_cfg)
    last_signal = signals.iloc[-1]
    
    # Extract signal components
    signal_value = last_signal['signal']  # 1=buy, -1=sell, 0=no trade
    rsi = last_signal['rsi']
    trend_side = last_signal['trend_side']  # 1=up, -1=down
    
    # Calculate confidence using calculate_signal_confidence from strategy_rsi.py
    confidence = calculate_signal_confidence(
        rsi=float(rsi),
        trend_side=int(trend_side),
        signal=int(signal_value),
        rsi_mode=strategy_cfg['rsi']['mode'],
        rsi_centerline=strategy_cfg['rsi']['centerline'],
        rsi_oversold=strategy_cfg['rsi']['oversold'],
        rsi_overbought=strategy_cfg['rsi']['overbought']
    )
    
    # Calculate strike difference and volatility
    strike_diff = abs(row.strike - last.Close)
    vol_ok = strike_diff <= 0.5 * last.ATR
    
    # Build recommendation + contract price
    if signal_value == 0 or not vol_ok:
        rec = "No trade"
        price = pd.NA
        confidence = 0.0  # No confidence for no-trade signals
    else:
        direction = "Buy" if signal_value == 1 else "Sell"
        price = 10 * (0.5 - (strike_diff / (2 * last.ATR)))
        rec = direction
    
    return pd.Series({
        "Date": pd.Timestamp.now().strftime("%d-%b-%y"),
        "Ticker": row.ticker,
        "Strike": row.strike,
        "EMA12": last.EMA12,
        "EMA26": last.EMA26,
        "MACD": last.MACD,
        "RSI": rsi,
        "ATR": last.ATR,
        "Recommendation": rec,
        "ContractPrice": price,
        "Confidence": confidence,
        "Trend": trend_side,
        "Momentum": rsi,  # RSI serves as momentum indicator
        "Signal": signal_value,
        "Volatility": strike_diff
    })

def generate_detailed_signals(per_ticker: dict[str, pd.DataFrame],
                              strikes_df: pd.DataFrame,
                              strategy_cfg: dict) -> pd.DataFrame:
    """
    Applies signal_detail_for_row to every strike with strategy config.
    Then applies guardrails to filter and limit recommendations.
    """
    signals = strikes_df.apply(
        lambda r: signal_detail_for_row(r, per_ticker, strategy_cfg),
        axis=1
    )
    
    # Apply guardrails using the strategy_rsi function
    signals['signal'] = signals['Signal']  # Map Signal column
    signals = apply_guardrails(
        signals, 
        strategy_cfg,
        signal_col='signal',
        confidence_col='Confidence'
    )
    signals.drop('signal', axis=1, inplace=True)  # Clean up
    
    return signals

## Define a function to run the Pipeline
1. Collect pricing data
2. Compute the technical indicators
3. Compute the trading signals
4. Compare with the day's Strike prices
5. Upload the analysis

In [ ]:
import pandas as pd
from datetime import date, datetime
from typing import List

def run_recommendation_pipeline(tickers: List,
                               bucket_name: str,
                               period: str,
                               interval: str,
                               mapping_file: str,
                               strategy_cfg: dict,
                               region: str = None) -> pd.DataFrame:
    """
    Fetch price, compute indicators, load strikes,
    and return a DataFrame of trade signals.
    """
    # Create S3 clients
    clients = create_s3_clients(region=region)
    public_s3 = clients["public"]
    private_s3 = clients["private"]
    s3_resource = clients["resource"]
    buckets = {
        "daily": get_bucket(s3_resource, bucket_name),
    }

    # Fetch price data for all tickers
    ticker_price_data = [
        (ticker, fetch_price(ticker, period, interval))
        for ticker in tickers
    ]

    # Compute technical indicators using config
    processed = {
        ticker: compute_indicators(df, strategy_cfg)
        for ticker, df in ticker_price_data
    }
    
    # Load strike prices and generate signals
    strikes_df = load_strikes(mapping_file, tickers_cfg)
    signals_df = generate_detailed_signals(processed, strikes_df, strategy_cfg)

    # Upload to S3
    today_str = date.today().strftime('%Y%m%d')
    s3_key = f"{RECS_PREFIX}/{today_str}.csv"

    # In run_recommendation_pipeline(), before uploading:
    ignals_df = generate_detailed_signals(processed, strikes_df, strategy_cfg)

   # In run_recommendation_pipeline(), add prints after generate_detailed_signals:
    signals_df = generate_detailed_signals(processed, strikes_df, strategy_cfg)

    # print(f"\n🔍 BEFORE showing trades:")
    # print(f"Total signals: {len(signals_df)}")
    # print(f"Recommendations != 'No trade': {len(signals_df[signals_df['Recommendation'] != 'No trade'])}")
    # if len(signals_df[signals_df['Recommendation'] != 'No trade']) > 0:
    #     trades = signals_df[signals_df['Recommendation'] != 'No trade']
    #     print(trades[['Ticker', 'Recommendation', 'Confidence', 'Signal', 'RSI']].head(10))

    upload_df_to_s3_with_validation(
        signals_df,
        bucket_name,
        s3_key,
        region=region
    )
    
    return signals_df

## Define a function to show interesting trades

In [ ]:
from tabulate import tabulate

def show_interesting_trades(df: pd.DataFrame) -> str:
    interesting_trades_df = df[
        ~df['Recommendation']
            .str.contains("No trade", case=False, na=False)
    ]
    
    if not interesting_trades_df.empty:
        columns_to_show = ["Date", "Ticker", "Recommendation", "Strike", "ContractPrice"]
        if 'Confidence' in interesting_trades_df.columns:
            columns_to_show.append("Confidence")
        
        display_df = interesting_trades_df[columns_to_show].copy()
        
        # Format confidence as percentage if present
        if 'Confidence' in display_df.columns:
            display_df['Confidence'] = display_df['Confidence'].apply(lambda x: f"{x:.1%}")
        
        print(
            tabulate(
                display_df,
                headers='keys',
                tablefmt='fancy_grid',
                showindex=False,
                maxcolwidths=200
            )
        )
        return 'Success'
    else:
        print("No trades recommended today")
        return 'Failed'

## Run recommendation pipeine

In [ ]:
import datetime as dt

# Load active tickers from config (only those marked as active: true)
TICKERS = {
    ticker_symbol 
    for ticker_symbol, info in tickers_cfg['tickers'].items() 
    if info.get('active', False)
}

# Track run start time
run_start = dt.datetime.now()
run_id = run_start.strftime("%Y%m%dT%H%M%S")

print(f"\n⚙️ Current Strategy Config:")
print(f"RSI mode: {strategy_cfg['rsi']['mode']}")
print(f"RSI Centerline: {strategy_cfg['rsi']['centerline']}")
print(f"RSI Oversold: {strategy_cfg['rsi']['oversold']}")
print(f"RSI Overbought: {strategy_cfg['rsi']['overbought']}")
print(f"Trend type: {strategy_cfg['trend']['type']}")
print(f"Confidence threshold: {strategy_cfg['guardrails']['confidence_threshold']}")

# Run the pipeline WITH strategy config
successful_run = show_interesting_trades(
    run_recommendation_pipeline(
        tickers=TICKERS,
        period="90d",
        interval="1d",
        bucket_name=BUCKET,
        mapping_file=MAPPING_FILE,
        strategy_cfg=strategy_cfg,  # ← Pass strategy config
        region=REGION
    )
)

# Create S3 client and log result
clients = create_s3_clients(region=REGION)
private_s3 = clients["private"]

append_runlog_s3(
    private_s3,
    BUCKET,
    RUNLOG_KEY,
    start_time=run_start,
    status=successful_run,
    files_processed=0,
    files_skipped=0,
    files_error=0,
    run_id=run_id,
    notes=f'Recommendation run - RSI:{strategy_cfg["rsi"]["mode"]}'
)

print(f"\n✅ Run complete: {run_id}")
print(f"   Status: {successful_run}")

## Test with Tickers

In [ ]:
# DEBUG: Test signal generation for multiple tickers
from tabulate import tabulate
import pandas as pd

# Test multiple tickers
# Check if diagnostic tests should run (from strategy config)
if strategy_cfg.get('notebook', {}).get('run_diagnostic_tests', False):
    # Get test tickers from config
    test_tickers = tickers_cfg.get('test_tickers', ['ES=F', 'NQ=F', 'GC=F', 'CL=F', 'EURUSD=X'])
    
    results = []

    print(f"\n⚙️ Current Strategy Config:")
    print(f"RSI mode: {strategy_cfg['rsi']['mode']}")
    print(f"RSI Centerline: {strategy_cfg['rsi']['centerline']}")
    print(f"RSI Oversold: {strategy_cfg['rsi']['oversold']}")
    print(f"RSI Overbought: {strategy_cfg['rsi']['overbought']}")
    print(f"Trend type: {strategy_cfg['trend']['type']}")
    print(f"Confidence threshold: {strategy_cfg['guardrails']['confidence_threshold']}")

    for ticker in test_tickers:
        try:
            # Fetch and process data
            test_data = fetch_price(ticker, period="90d", interval="1d")
            test_processed = compute_indicators(test_data, strategy_cfg)
            
            # Generate signals
            signals = generate_rsi_signals(test_processed['Close'], strategy_cfg)
            last_signal = signals.iloc[-1]
            
            # Calculate confidence
            confidence = calculate_signal_confidence(
                rsi=float(last_signal['rsi']),
                trend_side=int(last_signal['trend_side']),
                signal=int(last_signal['signal']),
                rsi_mode=strategy_cfg['rsi']['mode'],
                rsi_centerline=strategy_cfg['rsi']['centerline'],
                rsi_oversold=strategy_cfg['rsi']['oversold'],
                rsi_overbought=strategy_cfg['rsi']['overbought']
            )
            
            # Determine recommendation
            signal_value = int(last_signal['signal'])
            if signal_value == 1 and confidence >= strategy_cfg['guardrails']['confidence_threshold']:
                recommendation = "Buy"
            elif signal_value == -1 and confidence >= strategy_cfg['guardrails']['confidence_threshold']:
                recommendation = "Sell"
            else:
                recommendation = "No trade"
            
            results.append({
                'Ticker': ticker,
                'Close': f"{test_processed['Close'].iloc[-1]:.2f}",
                'RSI': f"{last_signal['rsi']:.2f}",
                'MACD': f"{test_processed['MACD'].iloc[-1]:.4f}",
                'MACD_Signal': f"{test_processed['Signal'].iloc[-1]:.4f}",
                'Trend': 'Up' if last_signal['trend_side'] == 1 else ('Down' if last_signal['trend_side'] == -1 else 'Neutral'),
                'Signal': signal_value,
                'Confidence': f"{confidence:.1%}",
                'Recommendation': recommendation
            })
        except Exception as e:
            results.append({
                'Ticker': ticker,
                'Close': 'ERROR',
                'RSI': str(e)[:50],
                'MACD': '-',
                'MACD_Signal': '-',
                'Trend': '-',
                'Signal': 0,
                'Confidence': '0%',
                'Recommendation': 'Error'
            })

    # Display results in a table
    print(f"\n📊 Signal Generation Test Results for {len(test_tickers)} Tickers:")
    print("="*120)
    results_df = pd.DataFrame(results)
    print(tabulate(results_df, headers='keys', tablefmt='fancy_grid', showindex=False))

    # Summary statistics
    buy_signals = sum(1 for r in results if r['Signal'] == 1)
    sell_signals = sum(1 for r in results if r['Signal'] == -1)
    no_signals = sum(1 for r in results if r['Signal'] == 0)
    buy_recs = sum(1 for r in results if r['Recommendation'] == 'Buy')
    sell_recs = sum(1 for r in results if r['Recommendation'] == 'Sell')

    print(f"\n📈 Summary:")
    print(f"   Total tickers tested: {len(test_tickers)}")
    print(f"   Buy signals (raw): {buy_signals}")
    print(f"   Sell signals (raw): {sell_signals}")
    print(f"   No signals: {no_signals}")
    print(f"   Buy recommendations (after confidence filter): {buy_recs}")
    print(f"   Sell recommendations (after confidence filter): {sell_recs}")# ... rest of test cell code ...
else:
    print("⏭️  Diagnostic tests skipped (run_diagnostic_tests: false in strategy.yaml)")